# Forecasting

## Summary

| #   | Method                                                  |mean squared error  |mean absolute error| |
|:----|:--------------------------------------------------------|-------------------:|------------------:|--:|
| 01  | Pandas moving average - pre-grouped by month (3 month)  | 234418077.64       | 10567.14   |2292.11|
| 2A  | Pandas moving average - pre-grouped by day (3 day)      | 916611.0          | 645.75    |   1605.19|
| 2B  | Pandas moving average - pre-grouped by day post-grouped month (3 day)|75117338.43| 3303.46| 47352.58 |
| 03  | SARIMA - Monthly - (0, 1, 0) (0, 1, 0, 12)          | 1428109073.12      | 17107.39         |
| 04  | SARIMA - Daily - (0, 1, 1) (1, 1, 1, 7)  |1910370.29|775.0|
| 05  | Exponential Smoothing - Monthly   |         |            |
| 06  | Exponential Smoothing - Daily   |         |            |

## References

* https://machinelearningmastery.com/moving-average-smoothing-for-time-series-forecasting-python/
* https://www.kaggle.com/code/carlmcbrideellis/time-series-a-simple-moving-average-ma-model
* https://medium.com/@josemarcialportilla/using-python-and-auto-arima-to-forecast-seasonal-time-series-90877adff03c
* https://machinelearningmastery.com/sarima-for-time-series-forecasting-in-python/
* https://machinelearningmastery.com/arima-for-time-series-forecasting-with-python/
* https://stackoverflow.com/questions/46146537/error-in-threading-sarimax-model


## Loading Libraries and datasets

In [373]:
# Importing data analytics libraries
import pandas as pd
import numpy as np

In [374]:
# Importing visualizing libraries
import matplotlib.pyplot as plt
import seaborn as sns
#Setting parameters for plot fig size
plt.rcParams["figure.figsize"] = (20,16)

In [375]:
# Importing performance metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [376]:
# statsmodels libraries
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [377]:
# Pyramid arima for autoarima
import pmdarima as pm

In [378]:
# Reading the cleaned and feature-engineered test dataset
df_test = pd.read_csv('../competitive-data-science-final-project/sales_test_with_target.csv')

In [ ]:
# Reading the cleaned and feature-engineered train dataset
df = pd.read_csv('../competitive-data-science-final-project/new_sales_train.csv')

## Grouping the dataframes to get one row each for the measured time unit

In [ ]:
# Grouping by date_block_num to get monthly item_cnt_day sum
df_monthly = df.groupby(df.date_block_num)[["item_cnt_day"]].sum()
df_monthly.tail()

In [ ]:
# Grouping the test data - Monthly
df_monthly_test = df_test.groupby(df_test.date_block_num)[["item_cnt_day"]].sum()

In [ ]:
# Grouping by date to get daily item_cnt_day sum
df_daily = df.groupby(df.date)[["item_cnt_day"]].sum()

In [ ]:
# Grouping the test data - Daily
df_daily_test = df_test.groupby(df_test.date)[["item_cnt_day"]].sum()

# 1. Pandas moving average - grouped by month - 3 month

In [ ]:
# Making a copy of the data frame
df_monthly_train = df_monthly.copy()

In [ ]:
# Adding rolling average column for 3 month rolling average
df_monthly_train["rolling_av3"] = df_monthly_train["item_cnt_day"].rolling(3).mean().round(2)

In [ ]:
# Adding the rolling average of month 32 as the actual for month 33 to predict the average for month 33 
df_monthly_train.loc[len(df_monthly_train)] = [df_monthly_train.rolling_av3[32], " "]

In [ ]:
# Predicting the rolling average for month 33 in a seperate column
df_monthly_train["rolling_av3_1"] = df_monthly_train["item_cnt_day"].rolling(3).mean().round(2)

In [ ]:
# Replacing the item_cnt_day value for month 33 with test data and calculating the MSE and MAE
actual_val = float(df_monthly_test.item_cnt_day.iloc[0])
df_monthly_train.at[33,"item_cnt_day"] = actual_val

In [ ]:
#mean squared error (y_true, y_pred)
mean_squared_error(df_monthly_train.item_cnt_day[2:], df_monthly_train.rolling_av3_1[2:]).round(2)

In [ ]:
#mean_absolute_error(y_true, y_pred)
mean_absolute_error(df_monthly_train.item_cnt_day[2:], df_monthly_train.rolling_av3_1[2:]).round(2)

In [ ]:
#Setting parameters for plot fig size
plt.rcParams["figure.figsize"] = (20,8)
plt.plot(df_monthly_train.item_cnt_day[2:])
plt.plot(df_monthly_train.rolling_av3_1[2:], color='red')
plt.title("Moving average (window 3 months) for monthly total sales",
         fontsize = 18)
plt.xlabel("Month Number")
plt.ylabel("Monthly item count (sum and rolling average)")
plt.legend(["Observed", "Predicted"])
plt.show()

In [ ]:
#mean_absolute_error(y_true, y_pred)
mean_absolute_error(df_monthly_train.item_cnt_day[-1:], df_monthly_train.rolling_av3_1[-1:]).round(2)

# 2A. Pandas moving average - daily - 3 day

In [ ]:
# Making a copy of the data frame
df_daily_train = df_daily.copy()

In [ ]:
# Adding rolling average column for 3 day rolling average
df_daily_train["rolling_av3"] = df_daily_train["item_cnt_day"].rolling(3).mean().round(2)

In [ ]:
length = len(df_daily_train)
length

In [ ]:
d = 1002
i = 0
while (d < 1033):
    df_daily_train["rolling_av3"] = df_daily_train["item_cnt_day"].rolling(3).mean().round(2)
    #df.at[4, 'B']
    df_daily_train.at[df_daily_test.index[i], "item_cnt_day"] = df_daily_train.rolling_av3[len(df_daily_train) - 1]
    d = d + 1
    i = i + 1
    #len(df_daily_train)
    #
df_daily_train["rolling_av3"] = df_daily_train["item_cnt_day"].rolling(3).mean().round(2)
print('Rolling average calculated')
# replace test data range of item_cnt_day with actual values
i = 0
while (i < len(df_daily_test)):
    df_daily_train.item_cnt_day[length + i] = df_daily_test.item_cnt_day[i]
    i = i + 1
print("item_cnt_day replaced by original values")

In [ ]:
#mean squared error (y_true, y_pred)
mean_squared_error(df_daily_train.item_cnt_day[2:], df_daily_train.rolling_av3[2:]).round(2)

In [ ]:
#mean_absolute_error(y_true, y_pred)
mean_absolute_error(df_daily_train.item_cnt_day[2:], df_daily_train.rolling_av3[2:]).round(2)

In [ ]:

plt.rcParams["figure.figsize"] = (20,8)

plt.plot(df_daily_train.item_cnt_day[2:])
plt.plot(df_daily_train.rolling_av3[2:], color='red')
plt.title("Moving average (window 3 days) for daily total sales",
         fontsize = 18)
plt.xlabel("Date")
plt.ylabel("Daily item count (sum and rolling average)")
plt.legend(["Observed", "Predicted"])
plt.show()

In [ ]:
mean_absolute_error(df_daily_train.item_cnt_day[-31:], df_daily_train.rolling_av3[-31:]).round(2)

# 2B. Pandas moving average - daily - 3 day (Grouped by Month)

In [ ]:
# Grouping the results by month
df_daily_train.head()
temp_monthly = df_daily_train.reset_index()

In [ ]:
type(temp_monthly.date[0])
temp_monthly['date']= pd.to_datetime(temp_monthly['date'])

In [ ]:
temp_monthly.index = temp_monthly['date']
temp_monthly = temp_monthly.groupby(pd.Grouper(freq='M'))[["item_cnt_day", "rolling_av3"]].sum()

In [ ]:
#mean squared error (y_true, y_pred)
mean_squared_error(temp_monthly.item_cnt_day, temp_monthly.rolling_av3).round(2)

In [ ]:
#mean_absolute_error(y_true, y_pred)
mean_absolute_error(temp_monthly.item_cnt_day, temp_monthly.rolling_av3).round(2)

In [ ]:
#Setting parameters for plot fig size
plt.rcParams["figure.figsize"] = (20,8)
plt.plot(temp_monthly.item_cnt_day)
plt.plot(temp_monthly.rolling_av3, color='red')
plt.title("Moving average (window 3 days) for Monthly total sales",
         fontsize = 18)
plt.xlabel("Date")
plt.ylabel("Monthly item count (sum and rolling average)")
plt.legend(["Observed", "Predicted"])
plt.show()

In [ ]:
mean_absolute_error(temp_monthly.item_cnt_day[-1:], temp_monthly.rolling_av3[-1:]).round(2)

# 3. ARIMA - Monthly

### 1. Rearranging the dataframe

In [ ]:
df_for_arima = df_daily.copy().reset_index()

In [ ]:
df_for_arima.date = pd.to_datetime(df_for_arima.date)
df_for_arima = df_for_arima.resample(rule='M', on='date')['item_cnt_day'].sum()
df_for_arima.head()

### 2. EDA

In [ ]:
# Visualizing data as is
plt.rcParams["figure.figsize"] = (20,8)
df_for_arima.plot()
plt.title("Monthly total item count",
         fontsize = 18)
plt.xlabel("Monthly")
plt.ylabel("Total item count of the month")

### 3. Augmented Dickey-Fuller test

Theory:

Using a statistical test to check if the difference between two samples of Gaussian random variables is real or a statistical fluke. 

Null Hypothesis (H0): 

* If failed to be rejected, it suggests the time series has a unit root, meaning it is non-stationary. 
* It has some time dependent structure.

Alternate Hypothesis (H1): 

* The null hypothesis is rejected; it suggests the time series does not have a unit root, meaning it is stationary. 
* It does not have time-dependent structure.

p-value
    
* p-value > 0.05: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
* p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.

In [ ]:

X = df_for_arima.values
result = adfuller(X)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
 print('\t%s: %.3f' % (key, value))


* **p-value is > 0.05**, the data fails to reject the null hypothesis.
    
* The data has a unit root and is **non-stationary**


### 4. Autocorrelation - statsmodels

In [ ]:
plot_acf(df_for_arima)
plt.show()

### 5. Seasonal decompose

In [ ]:
# Additive seasonal decompose
seasonal_decompose(df_for_arima, model='additive').plot()
plt.show()

In [ ]:
# Multiplicative seasonal decompose
seasonal_decompose(df_for_arima, model='multiplicative').plot()
plt.show()

In [ ]:
# There is a clear downward trend 
# There is also clear seasonality (The two peaks at 2014-01 and 2015-01)
# The volatility of the residual is reduced considerably in the multiplicative model compared to the additive model.
# Probably the multiplicative model is the better bet

# 4. SARIMA - Monthly

## 1. Figuring out the SARIMA settings with pmdarima/autoarima

In [ ]:
#!pip install pmdarima

In [ ]:
data = df_for_arima.copy()
data.head()

In [ ]:
data.tail()

In [ ]:
stepwise_fit = pm.auto_arima(data, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

## 2. Defining, fitting the model and forecasting

In [ ]:
# Defining the model
model = SARIMAX(data, order=(0,1,0), seasonal_order=(0,1,0, 12))
#fit model
results=model.fit(method='cg')
#https://stackoverflow.com/questions/46146537/error-in-threading-sarimax-model
print(results.summary())

In [ ]:
# one step forecast
yhat = results.forecast(steps=1)

In [ ]:
yhat

In [ ]:
predicted = results.predict(start=0, end=len(data))

In [ ]:
predicted.tail()

## 3. Creating a dataframe to measure against test data

In [ ]:
df_for_sarima_monthly = pd.DataFrame(data.copy())
df_for_sarima_monthly.tail()

In [ ]:
index_val = yhat.index[0]
index_val

In [ ]:
df_for_sarima_monthly.at[ index_val, 'item_cnt_day'] = df_monthly_test.item_cnt_day[33]
df_for_sarima_monthly.tail()

In [ ]:
df_for_sarima_monthly["Predicted"] = predicted
df_for_sarima_monthly.tail()

In [ ]:
df_for_sarima_monthly.head()

In [ ]:
#mean squared error (y_true, y_pred)
mean_squared_error(df_for_sarima_monthly.item_cnt_day[1:], df_for_sarima_monthly.Predicted[1:]).round(2)

In [ ]:
#mean_absolute_error(y_true, y_pred)
mean_absolute_error(df_for_sarima_monthly.item_cnt_day[1:], df_for_sarima_monthly.Predicted[1:]).round(2)


In [ ]:
# Plotting Predicted vs Original
plt.rcParams["figure.figsize"] = (20,8)
plt.plot(df_for_sarima_monthly.item_cnt_day[1:])
plt.plot(df_for_sarima_monthly.Predicted[1:], color='red')
plt.title("SARIMA for monthly total sales",
         fontsize = 18)
plt.xlabel("Month")
plt.ylabel("Monthly item count (actual and predicted)")
plt.legend(["Observed", "Predicted"])
plt.show()

# 5. ARIMA - Daily

In [ ]:
df_for_arima_daily = df_daily.copy().reset_index()
df_for_arima_daily.date = pd.to_datetime(df_for_arima_daily.date)
df_for_arima_daily = df_for_arima_daily.resample(rule='D', on='date')['item_cnt_day'].sum()
df_for_arima_daily.tail()

In [ ]:
# Plotting data as is
plt.rcParams["figure.figsize"] = (20,8)
df_for_arima_daily.plot()
plt.title("Daily total item count",
         fontsize = 18)
plt.xlabel("Day")
plt.ylabel("Total item count of the day")

## 1. Augmented Dickey-Fuller test

In [ ]:
# Augmented Dickey-Fuller test
X = df_for_arima_daily.values
result = adfuller(X)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
 print('\t%s: %.3f' % (key, value))

**p-value is < 0.05, the data rejects the null hypothesis.**

**The data does not have a unit root and is stationary**

## 2. Autocorrelation

In [ ]:
# Plot autocorrelation
plt.rcParams["figure.figsize"] = (20,8)
plot_acf(df_for_arima_daily)
plt.show()

In [ ]:
# Plot autocorrelation
plt.rcParams["figure.figsize"] = (20,8)
plot_pacf(df_for_arima_daily, method='ywm')
plt.show()

## 3. Seasonal decompose

In [ ]:
# Additive seasonal decompose
plt.rcParams["figure.figsize"] = (20,16)
seasonal_decompose(df_for_arima_daily, model='additive').plot()
plt.show()

In [ ]:
# Multiplicative seasonal decompose

seasonal_decompose(df_for_arima_daily, model='multiplicative').plot()
plt.show()

In [ ]:
# There is a slight downward trend
# There is clear seasonality
# Residual improved a lot when changed from additive model to multiplicative. 

# 6. SARIMA - Daily

In [ ]:
data = df_for_arima_daily.copy()

In [ ]:
stepwise_fit = pm.auto_arima(data, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=7,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
print("Complete")

In [ ]:
# Best model:  ARIMA(0,1,1)(1,1,1)[7] AIC=16412.118
model = SARIMAX(data, order=(0,1,1), seasonal_order=(1,1,1, 7))

In [ ]:
results=model.fit(method='cg')
print(results.summary())

In [ ]:
# one step forecast
yhat = results.forecast(steps=31)
yhat

In [ ]:
predicted = results.predict(start=0, end=len(data)+30)
predicted.tail()

In [ ]:
series_for_sarima_daily = predicted.copy()

In [ ]:
series_for_sarima_daily = pd.DataFrame(series_for_sarima_daily)

In [ ]:
list = []
for i in data:
    list.append(i)
print(len(list))
for i in df_daily_test.item_cnt_day:
    list.append(i)
print(len(list))

In [ ]:
series_for_sarima_daily["Original"] = list

In [ ]:
series_for_sarima_daily.tail()

In [ ]:
#mean squared error (y_true, y_pred)
mean_squared_error(series_for_sarima_daily.Original[1:], 
                   series_for_sarima_daily.predicted_mean[1:]).round(2)


In [ ]:
#mean_absolute_error(y_true, y_pred)
mean_absolute_error(series_for_sarima_daily.Original[1:], 
                   series_for_sarima_daily.predicted_mean[1:]).round(2)


In [ ]:
plt.rcParams["figure.figsize"] = (20,8)
plt.plot(series_for_sarima_daily.Original[1:])
plt.plot(series_for_sarima_daily.predicted_mean[1:], color='red')
plt.title("SARIMA for daily total sales",
         fontsize = 18)
plt.xlabel("Date")
plt.ylabel("Daily item count (actual and predicted)")
plt.legend(["Observed", "Predicted"])
plt.show()